In [ ]:
import numpy as np
import csv

from dave.tessPipeline.tessmastio import TessAstroqueryArchive
import dave.fileio.tpf as tpf
from astropy.timeseries import BoxLeastSquares
#from astropy.stats import BoxLeastSquares
from scipy.signal import savgol_filter
from astroquery.mast import Observations
import scipy.fftpack
from scipy import signal
from scipy.signal import savgol_filter
from scipy.stats import binned_statistic

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})

%matplotlib inline

In [ ]:
localPath = "/Users/vkostov/.eleanor"
ar = TessAstroqueryArchive(localPath)

planetNum = 1
tic, sector = 281341249, 7#260128333, 10#70763084, 2#3#30848598, 11#31507696, 11

if sector == 10:
    transit_ = '1579'
elif sector == 6:
    transit_ = '1484'

In [ ]:
def pull_and_save_2min_data(save_file, sectors_):
    
    tt, ff, ee, qq = [], [], [], []

    for ii in sectors_:
        dvt, hdr = ar.getLightcurve(tic, ii, ext=planetNum, header=True)
        flux = dvt['SAP_FLUX']#/np.nanmean(dvt['SAP_FLUX'])
        flux_err = dvt['SAP_FLUX_ERR']#/np.nanmean(dvt['SAP_FLUX'])
        time = dvt['TIME']
        quality_flags = dvt['QUALITY']
        
        tt = np.hstack((tt, time))
        ff = np.hstack((ff, flux))
        ee = np.hstack((ee, flux_err))
        qq = np.hstack((qq, quality_flags))

#flags = np.isnan(ff)

    if save_file == 'y':

        with open(str(tic) + '_SC_S'+str(np.array(sectors_)[0]), mode = 'w') as txt:
            txt_w = csv.writer(txt, delimiter = ' ')
    
#    for time, flux, err in zip(tt[~flags], ff[~flags], ee[~flags], qq[~flags]):
            for time, flux, err, qual in zip(tt, ff, ee, qq):
                txt_w.writerow([time, flux, err, qual])
    return tt, ff, ee, qq

In [ ]:
save_file = 'n'

if save_file == 'y':
    sectors_ = [10]#[4,5,6,7,8,9,10,11]
    time, flux, flux_err, quality = pull_and_save_2min_data(save_file, sectors_)
    flags = np.isnan(flux)

tpf_, hdr_tpf = ar.getTPF(tic, sector, ext=planetNum, header=True)
cube = tpf.getTargetPixelArrayFromFits(tpf_, hdr_tpf)

In [ ]:
#hdr_tpf

In [ ]:
#obs_query_string = "tess*-s%04i-%016i*" % (sector, tic)
#obsTable = Observations.query_criteria(mission="TESS", obs_id=obs_query_string)

In [ ]:
#try:
#    dvt, hdr = ar.getDvt(tic, sector, ext=planetNum, header=True)
#    flux = dvt['LC_DETREND']
#except:
#    dvt, hdr = ar.getLightcurve(tic, sector, ext=planetNum, header=True)
#    flux = dvt['SAP_FLUX']#dvt['PDCSAP_FLUX']

In [ ]:
dvt, hdr = ar.getLightcurve(tic, sector, ext=planetNum, header=True)
time = dvt['TIME']
flux = dvt['PDCSAP_FLUX']
cent1, cent2 = dvt['MOM_CENTR1'], dvt['MOM_CENTR2']
#    flux = dvt['SAP_FLUX']#dvt['PDCSAP_FLUX']

In [ ]:
#hdr

In [ ]:
#hdr_tpf

In [ ]:
plot_centroids_ = 'n'

if plot_centroids_ == 'y':

    fig = plt.subplots(figsize=(20, 10))

    grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)

    plt.subplot(311)
    plt.plot(time, flux, 'k.-')
#plt.xlim(min(time), max(time))
    plt.ylabel('SAPFLUX')

    plt.subplot(312)
    plt.plot(time, cent1, 'k.-')
    plt.ylabel('MOMCENT1')

    plt.subplot(313)
    plt.plot(time, cent2, 'k.-')
    plt.ylabel('MOMCENT2')

In [ ]:
fig = plt.subplots(figsize=(20, 10))

grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)

plt.subplot(grid[0, 0:2])
plt.plot(time, flux/np.nanmedian(flux), 'k.')#, alpha = 0.3)
plt.ylim(0.92,1.05)
#plt.xlim(min(time), max(time))
plt.title('SC Sector ' + str(sector) + ' SAPFLUX Lightcurve')#trans)

#plt.xlim(1325., 1335)
nn_ = 4
plt.xlim(time[0]+nn_*5, time[0]+5+nn_*5)

plt.subplot(grid[0, 2])
plt.imshow(np.nanmean(cube, axis = 0))#, origin = 'bottom')
plt.title('SC Sector ' + str(sector) + ' Aperture')#trans)

#plt.savefig('Sector_' + str(sector) + '_SAPFLUX_and_TPF.png')

In [ ]:
tt_ = time[tt]
ff_ = cube[:,3,2]

bins = len(tt_)/15.
#print((max(tt_) - min(tt_), tt_[1] - tt_[0], (cadence), tt_.shape, len(tt_)/30))
flags = np.isnan(ff_)
bin_means, bin_edges, binnumber = binned_statistic(tt_[~flags], ff_[~flags], statistic='mean', bins=bins)
bin_width = (bin_edges[1] - bin_edges[0])
bin_centers = bin_edges[1:] - bin_width/2

#plt.plot(time[tt], cc[:,3,2])
plt.plot(bin_centers, bin_means,'ro-')

In [ ]:
xx

In [ ]:
#RUN BLS
def run_bls_(x, y, tic, do_plot_, min_period, max_period_):
        
    x_ref = min(x)
    x -= min(x)
    
    period_grid = np.exp(np.linspace(np.log(min_period), np.log(max_period_), 50000))
    durations_ = 0.1+0.05*np.linspace(0,3,4)#0.02+0.02*np.linspace(0,11,12)#0.02+0.02*np.linspace(0,11,12)
    bls_results = []
    periods = []
    t0s = []
    depths = []
    durations = []

    # Compute the periodogram for each planet by iteratively masking out
    # transits from the higher signal to noise planets. Here we're assuming
    # that we know that there are exactly two planets.
    for i in range(1):
        bls = BoxLeastSquares(x, y)
        bls_power = bls.power(period_grid, durations_, oversample=20)
        bls_results.append(bls_power)
    
    # Save the highest peak as the planet candidate
        index = np.argmax(bls_power.power)
        periods.append(bls_power.period[index])
        t0s.append(bls_power.transit_time[index])
        depths.append(bls_power.depth[index])
        durations.append(bls_power.duration[index])
        
    # Mask the data points that are in transit for this candidate
        m = np.zeros(len(x), dtype=bool)
        m |= bls.transit_mask(x, periods[-1], 0.5, t0s[-1])
        
    # PLOT RESULTS   

    if do_plot_ == 'y':

        fig, axes = plt.subplots(len(bls_results), 2, figsize=(20, 10))
        
        for i in range(len(bls_results)):
    # Plot the lightcurve
            plt.subplot(2,3,(1,3))
            plt.plot(x+x_ref, y, 'k-')
            plt.title('detrended TESS lightcurve')
            plt.ylabel('[ppt]')
            plt.xlim((x + x_ref).min(), (x + x_ref).max())
    # Plot the periodogram 
            plt.subplot(234)
            plt.plot(np.log10(bls_results[i].period), bls_results[i].power, "k")
            plt.title('BLS periodogram')
    # Plot the folded transit (folded on the full period)            
            plt.subplot(235)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < p
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
            plt.ylim(-10.*depths[0],5*depths[0])
            plt.xlabel('Phase [days]')
#
#
    # Plot the folded transit (folded on 1-day)   
            plt.subplot(236)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < 1.0 
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
    # Overplot the phase binned light curve
            bins = np.linspace(-1., 1., 32)#
            denom, _ = np.histogram(x_fold, bins)
            num, _ = np.histogram(x_fold, bins, weights=y)
            denom[num == 0] = 1.0
#            plt.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="c", lw = 1)
            plt.title('TIC ' + str(tic) + "; P =" + str(np.round(p,2)) + "; to =" + str(round(x_ref+t0s[0],2)))
            plt.xlabel('Phase [days]')
            plt.ylim(-10.*depths[0],5*depths[0])
        
            plt.show()
#        plt.savefig("/Users/vkostov/Desktop/Ideas_etc/TESS/tesseract/ryan_LCs/pngs/" + str(tic) + ".png")
#        plt.close()

    return bls_power

In [ ]:
flags = np.isfinite(flux)

detrend_ = 'n'

if detrend_ == 'y':
    do_plot_ = 'y'
    time_detrend, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(time[~flags], flux[~flags], do_plot_, sector)
else:
    time_detrend, flux_detrend_ = time[~flags], 1e3*flux[~flags]

In [ ]:
min_period = 1.
max_period = round(0.8*(max(time_detrend) - min(time_detrend)))

do_plot_ = 'y'
bls_results = run_bls_(time_detrend, flux_detrend_, tic, do_plot_, min_period, max_period)
dur_ = 24*bls_results.duration[np.argmax(bls_results.power)]
print(tic,
      np.round(bls_results.period[np.argmax(bls_results.power)],2),
      np.round(time[~flags][0] + bls_results.transit_time[np.argmax(bls_results.power)],4),
      "%.2f" % dur_,
      int(1e3*np.round(bls_results.depth[np.argmax(bls_results.power)],0)))

In [ ]:
flags = np.isfinite(flux)

tt_, ff_ = time[flags], flux[flags]/np.nanmedian(flux[flags])
tt_1 = tt_#[tt_ < 1650.]
ff_1 = ff_#[tt_ < 1650.]
min_period = 5.
max_period = 15.#int((0.8*(max(tt_) - min(tt_))))
do_plot_ = 'y'
bls_results = run_bls_(tt_1, ff_1, tic, do_plot_, min_period, max_period)

In [ ]:
fig = plt.subplots(figsize=(20, 10))

flux_SAP = dvt['SAP_FLUX']

flux_PDC = dvt['PDCSAP_FLUX']

plt.subplot(211)
flags = np.isfinite(flux_SAP)
plt.plot(time[flags], flux_SAP[flags]/np.nanmedian(flux_SAP[flags]), 'k.')
plt.plot([1650.9,1650.9], [0.996,0.998], 'r--')
#plt.xlim(1650.,1653)
#plt.xlim(1630., 1640)
plt.title('SAPFLUX')

plt.subplot(212)
flags = np.isfinite(flux_PDC)
plt.plot(time[flags], flux_PDC[flags]/np.nanmedian(flux_PDC[flags]), 'k.')
plt.plot([1650.9,1650.9], [1.0005,1.001], 'r--')
#plt.xlim(1650.,1653)
#plt.xlim(1630., 1640)
plt.title('PDCSAPFLUX')

In [ ]:
tt_, ff_ = time[flags], flux[flags]

period_grid = np.exp(np.linspace(np.log(1.), np.log(100), 50000))
duration = 0.1+0.05*np.linspace(0,3,4)

bls_periods = np.zeros([4])
bls_t0s = np.zeros([4])
bls_depths = np.zeros([4])
bls_durations = np.zeros([4])

# planet 1
bls = BoxLeastSquares(tt_, ff_)
bls_power = bls.power(period_grid, duration, oversample=20)

bls_power0 = bls_power

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]
bls_duration = bls_power.duration[index]
transit_mask = bls.transit_mask(tt_, bls_period, 3*bls_duration, bls_t0)

bls_periods[0] = bls_period
bls_t0s[0] = bls_t0
bls_depths[0] = bls_depth
bls_durations[0] = bls_duration

# planet 2
bls = BoxLeastSquares(tt_[~transit_mask], ff_[~transit_mask])
bls_power = bls.power(period_grid, duration, oversample=20)

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]
bls_duration = bls_power.duration[index]
transit_mask = bls.transit_mask(tt_, bls_period, 3*bls_duration, bls_t0)

bls_periods[1] = bls_period
bls_t0s[1] = bls_t0
bls_depths[1] = bls_depth
bls_durations[1] = bls_duration

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 10))

# Plot the periodogram
ax = axes[0]
ax.axvline(np.log10(bls_periods[0]), color="C1", lw=5, alpha=0.8)
ax.axvline(np.log10(bls_periods[1]), color="C1", lw=5, alpha=0.8)
ax.axvline(np.log10(bls_periods[2]), color="C1", lw=5, alpha=0.8)
ax.axvline(np.log10(bls_periods[3]), color="C1", lw=5, alpha=0.8)
ax.plot(np.log10(bls_power0.period), bls_power0.power, "k")
ax.annotate("periods = {0:.4f}, {1:.4f}, {2:.4f}, {3:.4f} d".format(*bls_periods),
            (0, 1), xycoords="axes fraction",
            xytext=(5, -5), textcoords="offset points",
            va="top", ha="left", fontsize=12)
ax.set_ylabel("bls power")
ax.set_yticks([])
ax.set_xlim(np.log10(period_grid.min()), np.log10(period_grid.max()))
ax.set_xlabel("log10(period)")

# Plot the folded transits
for i in range(2):
    ax = axes[i+1]
    x_fold = (tt_ - bls_t0s[i] + 0.5*bls_periods[i])%bls_periods[i] - 0.5*bls_periods[i]
    m = np.abs(x_fold) < 0.4
    ax.plot(x_fold[m], ff_[m], ".k")
    inds = np.argsort(x_fold[m])
    ax.plot(x_fold[m][inds], bls.model(tt_[m], bls_periods[i], bls_durations[i], bls_t0s[i])[inds])

    # Overplot the phase binned light curve
    bins = np.linspace(-0.11, 0.11, 52)
    denom, _ = np.histogram(x_fold, bins)
    num, _ = np.histogram(x_fold, bins, weights=ff_)
    denom[num == 0] = 1.0
#    ax.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="C1")

    ax.set_xlim(-0.5, 0.5)
    ax.set_ylabel("de-trended flux [ppt]")
    ax.set_xlabel("time since transit");

In [ ]:
xxx

In [ ]:
# DETREND DATA

def detrend_data_(time, flux, do_plot_, sector):
    x = time
    y = flux/np.nanmedian(flux)
    mu = np.median(y)
    y = (y / mu - 1) * 1e3
    
# Identify outliers
    m = np.ones(len(y), dtype=bool)
    for i in range(10):
        y_prime = np.interp(x, x[m], y[m])
        smooth = savgol_filter(y_prime, 1021, polyorder=3)
        resid = y - smooth
        sigma = np.sqrt(np.mean(resid**2))
        m0 = np.abs(resid) < 3*sigma
        if m.sum() == m0.sum():
            m = m0
            break
        m = m0

# Only discard positive outliers
    m = (resid < 3*sigma) & (resid > -5*sigma)
# REMOVE DATA AROUND DATA GAPS/DISCONTINUITIES!!!!
    m[(x - np.min(x) >= 12.25) & (x - np.min(x) <= 15.25)] = False
#
# Shift the data so that it starts at t=0. This tends to make the fit
# better behaved since t0 covaries with period.
#    print(min(x), min(x[m]))
    x_ref = np.min(x[m])# if sector != 3 else np.min(x)
#    x -= x_ref

# Make sure that the data type is consistent
    x_bak_, y_bak_ = x, y
    x = np.ascontiguousarray(x[m], dtype=np.float64)
    y = np.ascontiguousarray(y[m], dtype=np.float64)
    smooth = np.ascontiguousarray(smooth[m], dtype=np.float64)

    x_detrend_ = x
    y_detrend_ = y - smooth
    
#    do_plot_= 'y'
    if do_plot_ == 'y':
        plt.figure(figsize=(20,5))
    
# Plot the data
#        plt.subplot(211)
        plt.plot(x_bak_+0*x_ref, y_bak_, "k", label="raw data")
        plt.plot(x+0*x_ref, smooth, lw = 3., label = 'sav-gol filter')
        plt.plot(x_bak_[~m]+0*x_ref, y_bak_[~m], "xr", label="outliers")
        plt.legend(fontsize=12)
#        plt.xlim(x_bak_.min(), x_bak_.max())
        plt.xlabel("time")
        plt.ylabel("flux")
#        plt.ylim(-25., 25.)
        
#        plt.subplot(212)
#        plt.plot(x_detrend_, y_detrend_, "k", label="data")
#        plt.legend(fontsize=12)
#        plt.xlim(x_detrend_.min(), x_detrend_.max())
#        plt.xlabel("time")
#        plt.ylabel("flux")
        plt.show()
    
    return x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m

In [ ]:
#RUN BLS
def run_bls_(x, y, tic, do_plot_, min_period, max_period_):
        
    x_ref = min(x)
    x -= min(x)
    
    period_grid = np.exp(np.linspace(np.log(min_period), np.log(max_period_), 50000))
    durations_ = 0.05+0.05*np.linspace(0,8,9)#0.05+0.05*np.linspace(0,4,5)
    bls_results = []
    periods = []
    t0s = []
    depths = []
    durations = []

    # Compute the periodogram for each planet by iteratively masking out
    # transits from the higher signal to noise planets. Here we're assuming
    # that we know that there are exactly two planets.
    for i in range(1):
        bls = BoxLeastSquares(x, y)
        bls_power = bls.power(period_grid, durations_, oversample=20)
        bls_results.append(bls_power)
    
    # Save the highest peak as the planet candidate
        index = np.argmax(bls_power.power)
        periods.append(bls_power.period[index])
        t0s.append(bls_power.transit_time[index])
        depths.append(bls_power.depth[index])
        durations.append(bls_power.duration[index])
        
    # Mask the data points that are in transit for this candidate
        m = np.zeros(len(x), dtype=bool)
        m |= bls.transit_mask(x, periods[-1], 0.5, t0s[-1])
        
    # PLOT RESULTS   

    if do_plot_ == 'y':

        fig, axes = plt.subplots(len(bls_results), 2, figsize=(20, 10))
        
        for i in range(len(bls_results)):
    # Plot the lightcurve
            plt.subplot(2,3,(1,3))
            plt.plot(x+x_ref, y, 'k-')
            plt.title('detrended TESS lightcurve')
            plt.ylabel('[ppt]')
            plt.xlim((x + x_ref).min(), (x + x_ref).max())
    # Plot the periodogram 
            plt.subplot(234)
            plt.plot(np.log10(bls_results[i].period), bls_results[i].power, "k")
            plt.title('BLS periodogram')
    # Plot the folded transit (folded on the full period)            
            plt.subplot(235)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < p
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
#            plt.ylim(-10.*depths[0],5*depths[0])
            plt.xlabel('Phase [days]')
#
#
    # Plot the folded transit (folded on 1-day)   
            plt.subplot(236)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < 1.0 
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
    # Overplot the phase binned light curve
            bins = np.linspace(-1., 1., 32)#
            denom, _ = np.histogram(x_fold, bins)
            num, _ = np.histogram(x_fold, bins, weights=y)
            denom[num == 0] = 1.0
#            plt.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="c", lw = 1)
            plt.title('TIC ' + str(tic) + "; P =" + str(np.round(p,2)) + "; to =" + str(round(x_ref+t0s[0],2)))
            plt.xlabel('Phase [days]')
#            plt.ylim(-10.*depths[0],5*depths[0])
        
            plt.show()

    return bls_power

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(time[~flags], flux[~flags], 'k')

In [ ]:
detrend_ = 'n'

if detrend_ == 'y':
    do_plot_ = 'y'
    time_detrend, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(time[~flags], flux[~flags], do_plot_, sector)
else:
    time_detrend, flux_detrend_ = time[~flags], 1e3*flux[~flags]

In [ ]:
min_period = 1.
max_period = round(0.8*(max(time_detrend) - min(time_detrend)))

do_plot_ = 'y'
bls_results = run_bls_(time_detrend, flux_detrend_, tic, do_plot_, min_period, max_period)
dur_ = 24*bls_results.duration[np.argmax(bls_results.power)]
print(tic,
      np.round(bls_results.period[np.argmax(bls_results.power)],2),
      np.round(time[~flags][0] + bls_results.transit_time[np.argmax(bls_results.power)],4),
      "%.2f" % dur_,
      int(1e3*np.round(bls_results.depth[np.argmax(bls_results.power)],0)))